# Import các thư viện

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

import sys
import os
import csv
import json
import pandas as pd

In [ ]:
# Thêm thư mục cha vào sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

from config.config_university_project import ConfigUniversityProject
from helper.logger_helper import LoggerSimple
from helper.error_helper import show_error_info
from helper.multithread_helper import multithread_helper
from helper.reader_helper import load_jsonl_from_gz, store_jsons_perline_in_file, convert_json_to_csv


### crawler_university_list.py
> Lấy danh sách các trường đại học (url, code, name)

In [ ]:
# Khởi tạo logger từ module LoggerSimple với tên của module hiện tại
logger = LoggerSimple(name=__name__).logger

# Hàm gửi yêu cầu GET đến URL và trả về nội dung HTML của trang web
def get_content_request(url='https://diemthi.tuyensinh247.com/diem-chuan.html'):
    return requests.get(url).content

# Hàm trích xuất dữ liệu trường đại học từ nội dung HTML
def extract_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    universities_data = []
    
    # Lặp qua từng phần tử (li) chứa thông tin của từng trường đại học trong danh sách
    for e_li in soup.select('#benchmarking > li'):
        e_a = e_li.find('a')
        url = 'https://diemthi.tuyensinh247.com' + e_a.get('href') if e_a.get('href') != '' else None
        university_code = e_a.find('strong').get_text()
        university_name = e_a.get_text().split('-')[-1].strip()
        
        # Tạo đối tượng JSON chứa thông tin của từng trường đại học
        university_obj = {
            'url': url,
            'university_code': university_code,
            'university_name': university_name
        }
        universities_data.append(university_obj)
    
    return universities_data

# Hàm chính, thực hiện lấy và xử lý dữ liệu trường đại học từ trang web
if __name__ == '__main__':
    # URL của trang web cần lấy dữ liệu
    url = 'https://diemthi.tuyensinh247.com/diem-chuan.html'
    
    # Gửi yêu cầu GET đến URL và lấy nội dung HTML
    content_html = get_content_request(url=url)
    
    # Trích xuất dữ liệu trường đại học từ nội dung HTML
    universities_data = extract_content(html=content_html)
    logger.info(universities_data)
    
    # Lấy đường dẫn lưu trữ dữ liệu trường đại học từ file cấu hình
    file_university_path = ConfigUniversityProject().file_university_path
    
    # Lưu dữ liệu trường đại học dưới dạng JSON line vào file
    store_jsons_perline_in_file(jsons_obj=universities_data, file_output_path=file_university_path)
    logger.info(f'stored data in {file_university_path}')
    
    # Lấy đường dẫn lưu trữ dữ liệu trường đại học dưới dạng CSV từ file cấu hình
    file_university_CSV_path = ConfigUniversityProject().file_university_path.replace('.gz', '.csv')
    
    # Chuyển đổi đối tượng JSON sang file CSV
    convert_json_to_csv(universities_data, file_university_CSV_path)    
    logger.info(f'stored data in {file_university_CSV_path}')


> Xem thử list đã crawl về

In [ ]:
university_df = pd.read_csv(f"./common/university.csv")
university_df

### crawler_diemchuan_2020.py
> crawl điểm chuẩn TỪNG NGÀNH CỦA TỪNG TRƯỜNG

In [ ]:
# Khởi tạo logger từ module LoggerSimple với tên của module hiện tại
logger = LoggerSimple(name=__name__).logger

# Hàm trích xuất dữ liệu điểm chuẩn từ trang web
def extract_data_diemchuan(url_diemchuan, university_meta, year=None):
    # Kiểm tra nếu year không được cung cấp, khởi tạo danh sách rỗng cho diemchuan_datas
    if year is None:
        diemchuan_datas = []

        # Lặp qua các năm từ 2019 đến 2014
        for year in [2019, 2018, 2017, 2016, 2015, 2014]:
            try:
                # Tạo URL mới bằng cách thêm năm vào URL gốc
                url_with_year = f'{url_diemchuan}?y={year}'
                logger.info(f'prepare extract {url_with_year}')
                
                # Gửi yêu cầu GET đến URL
                response = requests.get(url_with_year)
                
                # Kiểm tra nếu mã trạng thái của response là 200 (OK)
                if response.status_code == 200:
                    # Lấy nội dung HTML từ response
                    html = response.content
                    
                    # Sử dụng BeautifulSoup để phân tích HTML
                    soup = BeautifulSoup(html, 'html.parser')
                    
                    # Lấy ra phần tử table đầu tiên từ HTML
                    e_table = soup.select_one('table')
                    
                    # Lặp qua từng hàng (tr) trong table có class là 'bg_white'
                    for e_tr in e_table.select('.bg_white'):
                        # Lấy ra danh sách các phần tử td trong hàng (tr)
                        e_tds = e_tr.select('td')
                        
                        # Lấy thông tin mã ngành từ phần tử thứ hai (index 1) trong danh sách td
                        major_code = e_tds[1].get_text()
                        
                        # Lấy thông tin tên ngành từ phần tử thứ ba (index 2) trong danh sách td
                        major_name = e_tds[2].get_text()
                        
                        # Lấy danh sách các nhóm môn học từ phần tử thứ tư (index 3) trong danh sách td,
                        # tách chúng bằng dấu phẩy và loại bỏ khoảng trắng xung quanh mỗi nhóm môn học
                        subject_groups = [subject_group.strip() for subject_group in e_tds[3].get_text().split(',')]
                        
                        # Lấy điểm chuẩn từ phần tử thứ năm (index 4) trong danh sách td
                        point = e_tds[4].get_text()
                        
                        # Lấy ghi chú từ phần tử thứ sáu (index 5) trong danh sách td
                        note = e_tds[5].get_text()
                        
                        # Với mỗi nhóm môn học trong subject_groups, tạo một đối tượng diemchuan_obj
                        # và thêm vào danh sách diemchuan_datas
                        for subject_group in subject_groups:
                            diemchuan_obj = {
                                'major_code': major_code,
                                'major_name': major_name,
                                'subject_group': subject_group,
                                'point': point,
                                'note': note,
                                'year': year
                            }
                            logger.info(diemchuan_obj)
                            diemchuan_datas.append(diemchuan_obj)
                
                # Nếu mã trạng thái của response không phải là 200, ghi log lỗi
                else:
                    logger.info(f'{response.status_code} - {url_with_year}')
            
            # Bắt và ghi log lỗi nếu có bất kỳ ngoại lệ nào xảy ra trong quá trình trích xuất dữ liệu
            except Exception as e:
                logger.error(e)
                show_error_info(e)

        # Trả về một dictionary chứa diemchuan_datas và university_meta sau khi trích xuất xong
        return {'diemchuan_datas': diemchuan_datas, 'university_meta': university_meta}
    
    # Nếu year được cung cấp, trả về None
    return None


# Hàm thực thi lấy dữ liệu điểm chuẩn cho mỗi đối tượng trường đại học
def method_univerisy_data(university_obj):
    university_diemchuan_data = extract_data_diemchuan(
        url_diemchuan=university_obj.get('url'),
        university_meta=university_obj
    )
    return university_diemchuan_data


# Hàm lấy mã trường đại học từ trang web danh sách trường đại học
def get_university_code():
    url = f'https://diemthi.tuyensinh247.com/danh-sach-truong-dai-hoc-cao-dang.html'
    response = requests.get(url=url)
    # logger.info(response.text)
    lst_university_code = []
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lặp qua các dòng (tr) trong bảng để lấy thông tin mã trường và URL chi tiết
    for idx, e_tr in enumerate(soup.select('.code-shol > table > tr')):
        if idx > 0:
            lst_td = e_tr.select('td a')
            code = lst_td[0].get_text()
            url = lst_td[0].get('href')
            if code is not None and len(code) > 0:
                university_name = lst_td[1].get_text()
                lst_university_code.append({
                    'url': f'https://diemthi.tuyensinh247.com/diem-chuan/dai-hoc-cong-nghe-{code}.html',
                    'url_info': url,
                    'university_code': code,
                    'university_name': university_name
                })

    return lst_university_code


# Hàm chính, thực hiện lấy dữ liệu điểm chuẩn cho tất cả trường đại học
if __name__ == '__main__':
    # Lấy danh sách các trường đại học và mã trường từ trang web
    universities = get_university_code()
    
    # Thực thi lấy dữ liệu điểm chuẩn cho từng trường đại học đồng thời
    universities_diemchuan_data = multithread_helper(
        items=universities, method=method_univerisy_data,
        timeout_concurrent_by_second=360, debug=False,
        max_workers=20
    )

    # Lưu dữ liệu điểm chuẩn dưới dạng JSON line vào file
    file_university_diemchuan_path = ConfigUniversityProject().file_university_diemchuan_path
    store_jsons_perline_in_file(jsons_obj=universities_diemchuan_data, file_output_path=file_university_diemchuan_path)
    logger.info(f'stored file_university_diemchuan_path: {file_university_diemchuan_path}')

    # Lưu dữ liệu điểm chuẩn dưới dạng CSV
    file_university_diemchuan_CSV_path = ConfigUniversityProject().file_university_diemchuan_path.replace('.gz', '.csv')
    convert_json_to_csv(universities_diemchuan_data, file_university_diemchuan_CSV_path)    
    logger.info(f'stored file_university_diemchuan_path: {file_university_diemchuan_CSV_path}')


In [ ]:
# Tạo list để chứa các dataframe từ từng đơn vị dữ liệu
dfs = []

# Duyệt qua từng đơn vị dữ liệu
for unit in universities_diemchuan_data:
    # Tạo dataframe từ diemchuan_datas
    df = pd.DataFrame(unit['diemchuan_datas'])
    
    # Thêm các cột từ university_meta vào dataframe
    df['university_code'] = unit['university_meta']['university_code']
    df['university_name'] = unit['university_meta']['university_name']
    df['university_url'] = unit['university_meta']['url']
    df['university_url_info'] = unit['university_meta']['url_info']
    
    # Thêm dataframe vào list
    dfs.append(df)

# Ghép các dataframe trong list thành một dataframe lớn
final_df = pd.concat(dfs, ignore_index=True)


In [ ]:
final_df

In [ ]:
final_df.to_csv("diemchuan.csv",  encoding='utf-8-sig', index=False)